# Project 

Project is to practice Data Science concepts learned so far.

The project will include following tasks:
- Load dataset
- Clean up the data:
    - Encode replace missing values
    - Replace features values that appear incorrect
- Encode categorical variables
- Split dataset to Train/Test/Validation
- Add engineered features
- Train and tune ML model
- Provide final metrics using Validation dataset


## Dataset description
The dataset for Lab-2 is sample of the SBA dataset posted on Kaggle.
The dataset is from the U.S. Small Business Administration (SBA) The U.S. SBA was founded in 1953 on the principle of promoting and assisting small enterprises in the U.S. credit market (SBA Overview and History, US Small Business Administration (2015)). Small businesses have been a primary source of job creation in the United States; therefore, fostering small business formation and growth has social benefits by creating job opportunities and reducing unemployment. There have been many success stories of start-ups receiving SBA loan guarantees such as FedEx and Apple Computer. However, there have also been stories of small businesses and/or start-ups that have defaulted on their SBA-guaranteed loans.  
More info on the original dataset: https://www.kaggle.com/mirbektoktogaraev/should-this-loan-be-approved-or-denied

**Don't use original dataset, use only dataset provided with project requirements in eLearning**

## Preparation

Use dataset provided in the eLearning

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 1500)
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import itertools

#plots
#!pip install plotnine
import plotnine
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_decision_regions
import matplotlib.gridspec as gridspec
import seaborn as sns
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

#preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder
import bisect
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.exceptions import NotFittedError
from sklearn.model_selection import train_test_split

#models
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import tree

#metrics
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_curve 
from plotnine import *
from sklearn.metrics import confusion_matrix, accuracy_score

In [ ]:
"""
Created on Mon Mar 18 18:25:50 2019

Purpose: Analyze input Pandas DataFrame and return stats per column
Details: The function calculates levels for categorical variables and allows to analyze summarized information

To view wide table set following Pandas options:
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth',200)
"""
import pandas as pd
def describe_more(df,normalize_ind=False, weight_column=None, skip_columns=[], dropna=True):
    var = [] ; l = [] ; t = []; unq =[]; min_l = []; max_l = [];
    assert isinstance(skip_columns, list), "Argument skip_columns should be list"
    if weight_column is not None:
        if weight_column not in list(df.columns):
            raise AssertionError('weight_column is not a valid column name in the input DataFrame')
      
    for x in df:
        if x in skip_columns:
            pass
        else:
            var.append( x )
            uniq_counts = len(pd.value_counts(df[x],dropna=dropna))
            uniq_counts = len(pd.value_counts(df[x], dropna=dropna)[pd.value_counts(df[x],dropna=dropna)>0])
            l.append(uniq_counts)
            t.append( df[ x ].dtypes )
            min_l.append(df[x].apply(str).str.len().min())
            max_l.append(df[x].apply(str).str.len().max())
            if weight_column is not None and x not in skip_columns:
                df2 = df.groupby(x).agg({weight_column: 'sum'}).sort_values(weight_column, ascending=False)
                df2['authtrans_vts_cnt']=((df2[weight_column])/df2[weight_column].sum()).round(2)
                unq.append(df2.head(n=100).to_dict()[weight_column])
            else:
                df_cat_d = df[x].value_counts(normalize=normalize_ind,dropna=dropna).round(decimals=2)
                df_cat_d = df_cat_d[df_cat_d>0]
                #unq.append(df[x].value_counts().iloc[0:100].to_dict())
                unq.append(df_cat_d.iloc[0:100].to_dict())
            
    levels = pd.DataFrame( { 'A_Variable' : var , 'Levels' : l , 'Datatype' : t ,
                             'Min Length' : min_l,
                             'Max Length': max_l,
                             'Level_Values' : unq} )
    #levels.sort_values( by = 'Levels' , inplace = True )
    return levels

### Load data

In [ ]:
data = pd.read_csv('C:/Users/monta/OneDrive/Desktop/BUAN_AML/Projects/SBA_loans_project_1.csv')

In [ ]:
print("Data shape:", data.shape)

**Review dataset**

In [ ]:
desc_df = describe_more(data)
desc_df

## Dataset preparation and clean-up

Modify and clean-up the dataset as following:
- Replace encode Na/Null values
- Convert the strings styled as '$XXXX.XX' to float values. Columns = ['DisbursementGross', 'BalanceGross', 'GrAppv', 'SBA_Appv']
- Convert MIS_Status to 0/1. Make value "CHGOFF" as 1

#### Here I consolidated the preprocessing into a function. This makes it easier to save and to also
#### replicate the steps. I dropped columns with missing values. This way any missing values in a new dataset
#### would intentionally count against the model.

### Engineered features

### 2 types of features, 2 new columns for each

#### 1st new column: Existing_Business_Job 
#### 2nd new column: New_Business_Job
New Exist(1 = Exist, 2 = New Business) : CreateJob This can give insight to expansion, or if creating jobs contributed to their ability to pay off.  

In [ ]:
def preprocessing_for_scoring(df):
    from sklearn.metrics import confusion_matrix
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import classification_report
    import pandas as pd
    from sklearn.model_selection import train_test_split

    """"
    Function is to clean the dataset before target encoding and scoring
    
    FLOW: Create function to help convert
    define col
    drop nas
    modify cols to make easier model scoring
    
    Then create Engineered features
    """
    def convert3(x):
        import pandas as pd
        return pd.to_numeric(x,errors='ignore', downcast="float")

    cols_to_strip = ['DisbursementGross','BalanceGross','GrAppv','SBA_Appv']

    df['MIS_Status'] = df['MIS_Status'].replace({'P I F': 0, 'CHGOFF': 1})
    
    df.dropna(inplace=True)
    for col in cols_to_strip:
        df[col] = df[col].str.replace('[$,]', '', regex=True)
        df[col] = df[col].apply(convert3)

    df = df[(df['RevLineCr'] == 'Y') | (df['RevLineCr'] == 'N')]
    df = df[(df['LowDoc'] == 'Y') | (df['LowDoc'] == 'N')]
    df = df.astype({'Zip': 'object', 'UrbanRural': 'object','NewExist':'int64'})
    df= df[df.NewExist != 0]
    df = df.astype({'NewExist':'object'})    
    df= df[(df['NewExist'] == 1) | (df['NewExist'] == 2)]

    ##Engineered Features------------------------------------
    ## New Exist(1 = Exist, 2 = New Business) : CreateJob This can give insight to expansion, or simply trying to survive
    def myfunc(NewExist, CreateJob):
        if NewExist==2 and CreateJob!=0:
            Existing_Business_Job = 1
        else:
            Existing_Business_Job = 0
        return Existing_Business_Job
    df['Existing_Business_Job'] = df.apply(lambda x: myfunc(x['NewExist'], x['CreateJob']), axis=1)

    def myfunc(NewExist, CreateJob):
        if NewExist==1 and CreateJob!=0:
            New_Business_Job = 1
        else:
            New_Business_Job = 0
        return New_Business_Job
    df['New_Business_Job'] = df.apply(lambda x: myfunc(x['NewExist'], x['CreateJob']), axis=1)
    ## Term( loan term in months) : RevLineCr
    ## For those with short or long term amounts, if used to see if they defaulted, 
    #we can see predict if a revLineCr does more harm than good.
    ## Because revolving line of credit empowers the borrower, i am curious to see 
    #if it can help probability of defaulting
    def myfunc(Term, RevLineCr):
        if Term>=96 and RevLineCr == 'Y':
            Long_Term_RevCr = 1
        else:
            Long_Term_RevCr = 0
        return Long_Term_RevCr
    df['Long_Term_RevCr'] = df.apply(lambda x: myfunc(x['Term'], x['RevLineCr']), axis=1)

    def myfunc(Term, RevLineCr):
        if Term <96 and RevLineCr == 'Y':
            Short_Term_RevCr = 1
        else:
            Short_Term_RevCr = 0
        return Short_Term_RevCr
    df['Short_Term_RevCr'] = df.apply(lambda x: myfunc(x['Term'], x['RevLineCr']), axis=1)
    
    ###----------------------Dummies/OHE
    import pandas as pd
    import numpy as np
    BalanceGross_dummies = pd.get_dummies(df.BalanceGross,prefix='BalanceGross')
    df = pd.concat([df, BalanceGross_dummies], axis=1)

    LowDoc_dummies = pd.get_dummies(df.LowDoc,prefix='LowDoc')
    df = pd.concat([df, LowDoc_dummies], axis=1)
    
    UrbanRural_dummies = pd.get_dummies(df.UrbanRural,prefix='UrbanRural')
    df = pd.concat([df, UrbanRural_dummies], axis=1)


    #OHE/pandas dummies
    df.drop('LowDoc', axis=1, inplace=True)
    df.drop('UrbanRural', axis=1, inplace=True)
    df.drop('BalanceGross', axis=1, inplace=True)
    ## used to make features
    df.drop('Term', axis=1, inplace=True)
    df.drop('NewExist', axis=1, inplace=True)
    df.drop('RevLineCr', axis=1, inplace=True)
    df.drop('CreateJob', axis=1, inplace=True)

    return (df)

In [ ]:
df = preprocessing_for_scoring(data)

## Categorical variables encoding

Encode categorical variables using either one of the techniques below. Don't use LabelEncoder.
- One-hot-encoder for variables with less than 10 valid values. Name your new columns "Original_name"_valid_value
- (If using sklearn) Target encoder from the following library: https://contrib.scikit-learn.org/category_encoders/index.html . Name your new column "Original_name"_trg
- (If using H2O) Use H2O target encoder


Example of use for target encoder:
```
import category_encoders as ce

encoder = ce.TargetEncoder(cols=[...])

encoder.fit(X, y)
X_cleaned = encoder.transform(X_dirty)
```

In [ ]:
for col in df.columns:
    obj= {}
    if df[col].dtype == 'object':
        obj[col] = df[col].nunique()
        for key, value in obj.items():
            print('obj_col',key,': ', value)
print()
for col in df.columns:
    integer={}
    if df[col].dtype == 'int64':
        integer[col] = df[col].nunique()
        for key, value in integer.items():
            print('int_col',key,': ', value)
print()
for col in df.columns:
    float_data={}
    if df[col].dtype == 'float32':
        float_data[col] = df[col].nunique()
        for key, value in float_data.items():
            print('float_col',key,': ', value)
            
print()
for col in df.columns:
    float_data={}
    if df[col].dtype == 'float64':
        float_data[col] = df[col].nunique()
        for key, value in float_data.items():
            print('float_col',key,': ', value)

### Due to dropping the columns that were used to create new features, I placed that in the function above. 
### This changed those that needed to be target encoded and scaled.

## Target Encoding via sklearn

In [ ]:
# !pip install category_encoders
import category_encoders as ce
from sklearn.model_selection import train_test_split

x = df.drop(columns = ['MIS_Status'], axis = 1).copy()
y = df[['MIS_Status']]
te = ce.TargetEncoder()

# Splitting into train/test
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size = .30,
                                                    train_size = .7,
                                                    random_state=9999)
suffix = 'trg'
cols_te =['City','Zip','State','Bank','BankState']

x_train[cols_te] = te.fit_transform(x_train[cols_te], y_train)
x_test[cols_te] = te.transform(x_test[cols_te], y_test)

x_train.columns= x_train.columns.map(lambda x : x +'_trg' if x in cols_te else x)
x_test.columns = x_test.columns.map(lambda x : x +'_trg' if x in cols_te else x)

## MinMax Scaler

In [ ]:
from sklearn.preprocessing import MinMaxScaler

cols_to_scale = ['NAICS','NoEmp','RetainedJob','FranchiseCode','DisbursementGross','GrAppv','SBA_Appv']
sc = MinMaxScaler()

suffix = '_sc'

x_train[cols_to_scale] = sc.fit_transform(x_train[cols_to_scale])
x_test[cols_to_scale] = sc.transform(x_test[cols_to_scale])

x_train.columns= x_train.columns.map(lambda x : x +'_sc' if x in cols_to_scale else x)
x_test.columns = x_test.columns.map(lambda x : x +'_sc' if x in cols_to_scale else x)

# Model Training / ## Model Tuning

Depending on the model of your choice, you might need to use appropriate scaler for numerical variables.

Train at least two types of models from the below list.
If you use sklearn libraries:
- Logistic regression
- SVM
- Decision Tree

If you use H2O libraries:
- GLM
- SVM
- Naïve Bayes Classifier

Choose one model from the above list. You should provide reasoning on why you have picked the model over others. Perform tuning for the selected model:
- Hyper-parameter tuning. Your hyper-parameter search space should have at least 50 combinations.
- To avoid overfitting and provide you with reasonable estimate of model performance on hold-out dataset, you will need to split your dataset as following:
    - Train, will be used to train model
    - Validation, will be used to validate model each round of training
    - Testing, will be used to provide final performance metrics, used only once on the final model
- Feature engineering. You should add at least two engineered features.  For example, add feature which is combination of two features.
- If your model returns probability, calculate probability threshold to maximize F1. 

### SVC Model. Did the hyper parameter tuning sooner to give a cleaner flow. 

In [ ]:
from sklearn.svm import SVC

X_train_svm = x_train.copy()
Y_train_svm = y_train.copy()

X_test_svm= x_test.copy()
Y_test_svm = y_test.copy()

# ## We set the parameters to use for the best estimator. 
# svm = SVC(kernel='poly', max_iter=700, verbose=False)
# degree = [2,3,4,5,7]
# parameters = dict(degree = degree)

# ##utilize the best estimator to find the best degree
# GridSearch = GridSearchCV(svm, parameters)
# Fitted_Model = GridSearch.fit(X_train_svm, Y_train_svm)
# print(Fitted_Model.best_estimator_.get_params()['degree'])

In [ ]:
SVM = SVC(kernel='poly', degree=4, max_iter=700, verbose=False)

SVM.fit(X_train_svm,Y_train_svm)
SVM_y_pred = SVM.predict(X_test_svm)

labels = np.unique(Y_test_svm)
confusion = metrics.confusion_matrix(Y_test_svm, SVM_y_pred, labels = np.unique(Y_test_svm))
print('Confusion matrix')
print(pd.DataFrame(confusion, index=labels, columns=labels))
print()
print('Classification Report')
print(classification_report(Y_test_svm, SVM_y_pred))

In [ ]:
print("Target Column:", y_train.sum()/len(y_train)) ## unbalanced

### Decision Tree - chosen model

### I decided chose the DT model due to the poor scoring for the SVM.
### The poor scoring may be due to the imbalance of the dataset. 
### Also, the preprocssing could have impacted the model to make the classification less accurate. 
### The Dt will deal better with the collinearity overall due to some features being related. 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

X_train_dt = x_train.copy()
Y_train_dt = y_train.copy()

X_test_dt = x_test.copy()
Y_test_dt = y_test.copy()

dtc = DecisionTreeClassifier()

pipe = Pipeline(steps=[('dtc', dtc)])

max_depth = [2,4,6,8,10,12,100]
ccp_alpha = [0,0.0005, 0.001, 0.0015, 0.002, 0.0025]
criterion = ['entropy']

parameters = dict(dtc__max_depth=max_depth,
                  dtc__ccp_alpha =ccp_alpha,
                  dtc__criterion=criterion)

dtc = GridSearchCV(dtc, parameters)
GridSearch = GridSearchCV(pipe, parameters)
DTC_Model = GridSearch.fit(X_train_dt, Y_train_dt)

print('Best max_depth:', DTC_Model.best_estimator_.get_params()['dtc__max_depth'])
print('Best ccp_alpha:', DTC_Model.best_estimator_.get_params()['dtc__ccp_alpha'])
print('Best Criterion:', DTC_Model.best_estimator_.get_params()['dtc__criterion'])

In [ ]:
X_train_dt2 = x_train.copy()
Y_train_dt2 = y_train.copy()

X_test_dt2 = x_test.copy()
Y_test_dt2 = y_test.copy()

dtc3 = DecisionTreeClassifier(random_state =0, max_depth =12, ccp_alpha= 0, criterion ='entropy')
dtc3.fit(X_train_dt2,Y_train_dt2)
pred_train_dt = dtc3.predict(X_train_dt2)
pred_test_dt = dtc3.predict(X_test_dt2)

##We see that the weighted f1score is 
labels = np.unique(Y_test_dt2)

##Train
print('Decision Tree Model 2')
confusion = metrics.confusion_matrix(Y_train_dt2, pred_train_dt, labels = np.unique(Y_test_dt2))
print('Confusion matrix')
print(pd.DataFrame(confusion, index=labels, columns=labels))
print(classification_report(Y_train_dt2, pred_train_dt))
print()

##Test
print('Decision Tree Model 2')
confusion = metrics.confusion_matrix(Y_test_dt2, pred_test_dt, labels = np.unique(Y_test_dt2))
print('Confusion matrix')
print(pd.DataFrame(confusion, index=labels, columns=labels))
print(classification_report(Y_test_dt2, pred_test_dt))

## Save all artifacts

Save all artifacts needed for scoring function:
- Trained model
- Encoders

You should restart your Kernel now to properly test scoring function

In [ ]:
# !pip install cloudpickle
import pickle
import cloudpickle
from pickle import dump

with open('preprocessing_for_scoring', 'wb') as preprocessing_for_scoring_file:
    cloudpickle.dump(preprocessing_for_scoring, preprocessing_for_scoring_file)

#Saving the MinMax Scaler
dump(te, open('te.pkl', 'wb'))
with open('te', 'wb') as te_file:
    cloudpickle.dump(te, te_file)
    
#Saving the MinMax Scaler
dump(sc, open('sc.pkl', 'wb'))
with open('sc', 'wb') as sc_file:
    cloudpickle.dump(sc, sc_file)

#Saving the Decision Tree
dump(dtc3, open('dtc3.pkl', 'wb'))
with open('dtc3', 'wb') as dtc3_file:
    cloudpickle.dump(dtc3, dtc3_file)

# -----------------Restart Kernel Here---------------

## Model Scoring

Write function that will load artifacts from above, transform and score on a new dataset.
Your function should return Python list of labels. For example: [0,1,0,1,1,0,0]


In [1]:
def final_scorer(df):
    import pandas as pd
    import cloudpickle
    import numpy as np
    import pandas as pd
    import pickle
    from pickle import dump
    from pickle import load
    from sklearn import metrics
    from sklearn.metrics import accuracy_score, f1_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import precision_recall_curve 
    from sklearn.metrics import confusion_matrix, accuracy_score

#     """
#     Function to score input dataset.
    
#     Input: dataset in Pandas DataFrame format
#     Output: Python list of labels in the same order as input records
    
#     Flow:
        
#         - Load Df_cleaning_Preprocessor
#         - load target encoder
#         - Transform dataset
#         - load model
#         - Score dataset
#         - Return labels
#     """
    cols_to_scale = ['NAICS','NoEmp','RetainedJob','FranchiseCode','DisbursementGross','GrAppv','SBA_Appv']
    cols_te =['City','Zip','State','Bank','BankState']
    
    print('Cleaning Df....')
    with open(r"preprocessing_for_scoring", "rb") as preprocessing_for_scoring_file:
        preprocessing_for_scoring = cloudpickle.load(preprocessing_for_scoring_file)
    df = preprocessing_for_scoring(df)
    preprocessing_for_scoring_file.close()
    
    print('Target Encoding.....')
    with open(r"te", "rb") as te_file:
        te = cloudpickle.load(te_file)
    df[cols_te] = te.transform(df[cols_te])
    df.columns= df.columns.map(lambda x : x +'_trg' if x in cols_te else x)

    
    print('Scaling columns....')
    scaler = load(open('sc.pkl', 'rb'))
    with open(r"sc", "rb") as sc_file:
        sc = cloudpickle.load(sc_file)
    df[cols_to_scale] = sc.transform(df[cols_to_scale])
    df.columns= df.columns.map(lambda x : x +'_sc' if x in cols_to_scale else x)
    
    print('Loading Decision tree model....')
    decision_tree = load(open('dtc3.pkl', 'rb'))
    with open(r"dtc3", "rb") as dtc3_file:
        dtc3 = cloudpickle.load(dtc3_file)
    
    Y = df['MIS_Status']
    X = df.drop(['MIS_Status'], axis = 1, inplace = True)
    pred_dt = dtc3.predict(df)
    labels = np.unique(Y)

    
    print('Scoring your df ......')
    Confusion_Matrix = metrics.confusion_matrix(Y, pred_dt, labels = np.unique(Y))
    print('Decision Tree Model')
    print(Confusion_Matrix,pd.DataFrame(Confusion_Matrix, index=labels, columns=labels))
    print('Classification Report:')
    return print(classification_report(Y,pred_dt,labels= np.unique(Y)))

In [2]:
import pandas as pd

data = pd.read_csv('C:/Users/monta/OneDrive/Desktop/BUAN_AML/Projects/SBA_loans_project_1.csv')
final_scorer(data)

Cleaning Df....
Target Encoding.....
Scaling columns....
Loading Decision tree model....
Scoring your df ......
Decision Tree Model
[[431000  20078]
 [ 59738  40074]]         0.0    1.0
0.0  431000  20078
1.0   59738  40074
Classification Report:
              precision    recall  f1-score   support

         0.0       0.88      0.96      0.92    451078
         1.0       0.67      0.40      0.50     99812

    accuracy                           0.86    550890
   macro avg       0.77      0.68      0.71    550890
weighted avg       0.84      0.86      0.84    550890

